In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [2]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews-2.csv")
Hotel_City_Reviews=pd.read_csv("Hotel_Info_Country_City.csv", usecols=['Hotel_Name','City'])
Hotel_City_Reviews.head()

,Hotel_Name,City
0,Hotel Arena,Amsterdam
1,K K Hotel George,London
2,Apex Temple Court Hotel,London
3,The Park Grand London Paddington,London
4,Monhotel Lounge SPA,Paris


In [3]:
Amsterdam=Hotel_City_Reviews.loc[Hotel_City_Reviews['City']=='Amsterdam']
Amsterdam.Hotel_Name.unique()

array(['Hotel Arena', 'Morgan Mees', 'Sir Albert Hotel',
       'Corendon Vitality Hotel Amsterdam',
       'Hampshire Hotel Rembrandt Square Amsterdam',
       'Pillows Anna van den Vondel Amsterdam',
       'Best Western Delphi Hotel', 'Apollofirst Boutique Hotel',
       'Hilton Amsterdam', 'Apollo Hotel Amsterdam', 'Element Amsterdam',
       'Urban Lodge Hotel', 'Hotel De Hallen', 'Chasse Hotel',
       'NH Collection Amsterdam Grand Hotel Krasnapolsky',
       'Park Plaza Victoria Amsterdam', 'Hotel Amsterdam De Roode Leeuw',
       'Swiss tel Amsterdam', 'Holiday Inn Amsterdam', 'Jaz Amsterdam',
       'Best Western Premier Hotel Couture', 'Bilderberg Garden Hotel',
       'NH Amsterdam Noord', 'Grand Hotel Downtown', 'Amadi Park Hotel',
       'Novotel Amsterdam City', 'Savoy Hotel Amsterdam',
       'Hotel Okura Amsterdam', 'Hotel V Fizeaustraat',
       'Crowne Plaza Amsterdam South', 'WestCord Fashion Hotel Amsterdam',
       'The Hoxton Amsterdam', 'Ambassade Hotel',
      

In [4]:
Hotel_Amsterdam = Hotel_Reviews[Hotel_Reviews['Hotel_Name'].isin(Amsterdam.Hotel_Name.unique())]
len(Hotel_Amsterdam)

57214

In [5]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/LiverJoker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in Hotel_Amsterdam.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in Hotel_Amsterdam['Negative_Review'][i]) and "No Positive" in Hotel_Reviews['Positive_Review'][i]:
            continue
       elif "No Negative" in Hotel_Amsterdam['Negative_Review'][i]:
            reviewP = Hotel_Amsterdam['Positive_Review'][i]
       else:
            reviewN = Hotel_Amsterdam['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [word for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [word for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [7]:
len(corpusP)

14234

In [8]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [9]:
from functools import reduce

In [10]:
bowA = []
for line in corpusP:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

14234

In [ ]:
wordSet=set(bowAnew)


In [12]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [ ]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)


In [14]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PositiveReviews = PosRev.iloc[0:10, :]
PositiveReviews

,0
staff,1.222814
location,0.900219
hotel,0.854905
great,0.692240
room,0.687680
friendly,0.643352
helpful,0.592989
good,0.528664
breakfast,0.515401
clean,0.503197


In [ ]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidf = {x:y for x,y in tfidf.items() if y!=0}
    tfidfBowA = np.append(tfidfBowA, tfidf)


#sum(tfidfBowA[1].values())
tfidfBowA[0]

{'amenities': 0.08348132785166741,
 'amsterdam': 0.03834649962967877,
 'anyone': 0.08204666020931393,
 'aside': 0.1407974851952265,
 'back': 0.05434133643861843,
 'background': 0.1407974851952265,
 'bit': 0.1385679910029648,
 'check': 0.06270254986462136,
 'checkout': 0.10424851546516231,
 'complaints': 0.09948495050563312,
 'confusing': 0.26872306219654307,
 'deposit': 0.11373799166763246,
 'even': 0.06247250212965998,
 'excellent': 0.03257795231500192,
 'fantastic': 0.05042078457931065,
 'firstly': 0.12625318786389073,
 'food': 0.04994692802137921,
 'great': 0.06878979766091341,
 'hotel': 0.01856698480521391,
 'however': 0.08553970599942189,
 'lacking': 0.15179982139459072,
 'location': 0.017632392856135083,
 'make': 0.07174545445936459,
 'minor': 0.23050170332905304,
 'new': 0.067541603975524,
 'offer': 0.09137660727125232,
 'one': 0.056578606125595766,
 'option': 0.09629970137131327,
 'original': 0.1077904765971339,
 'payment': 0.12625318786389073,
 'payments': 0.1407974851952265,


In [16]:
OneHotList=[]
for i in range(0, len(tfidfBowA), 1):
    tempList=[]
    for key in tfidfBowA[i]:
        if tfidfBowA[i][key]>.05:
               tempList.append(key)
    OneHotList.append(tempList)

In [17]:
import collections 
import mlxtend

In [18]:
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori

In [19]:
oht = OnehotTransactions()
oht_ary = oht.fit(OneHotList).transform(OneHotList)  
df = pd.DataFrame(oht_ary, columns=oht.columns_)  
df

,aa,ab,abd,ability,abit,able,aboutin,abroad,absence,absloutley,...,ziggy,zillion,zipper,zone,zoo,zsa,zuid,zuidoost,zurich,zusje
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.069903,[amazing],1
1,0.083041,[amsterdam],1
2,0.092174,[bed],1
3,0.133413,[breakfast],1
4,0.070957,[city],1
5,0.128636,[clean],1
6,0.057187,[close],1
7,0.121329,[comfortable],1
8,0.075313,[everything],1
9,0.125334,[excellent],1
